# **Linear programming**

The current simplex algorithm is designed for the linear programming problem in the standard form, where we consider $x \in \mathbb{R}^n$, 
$A \in \mathbb{R}^{m \times n}$ y $b \in \mathbb{R}^m$
\begin{equation}
\left\{
\begin{array}{rl}
\min & c^T x \\
s.t. & Ax \leq b 
\end{array}
\right.
\end{equation}

basic bases
\begin{equation}
B = \left[
\begin{array}{c|c|c|c|c}
A_{\beta(1)} & \cdots & A_{j} & \cdots & A_{\beta(m)}
\end{array}
\right]
\end{equation}

In a block representation the simplex table has the following form
\begin{equation}
X
= \left[
\begin{array}{cc}
-c_B^T B^{-1}b & c^T - c^T_B B^{-1} A \\
B^{-1} b & B^{-1} A
\end{array}
\right]
\end{equation}
$\DeclareMathOperator*{\argmin}{argmin}$

In [1]:
import numpy as np
from scipy import linalg as la

---
## **Simplex algorithm**
1. When all the constraints are inequality constraints we can define a initial basic base associated to the additional slack variables 
$\beta = \{n+1, \ldots, n+m\}$
\begin{equation}
X
= \left[
\begin{array}{ccc}
0 & c^T & 0 \\
b & A & I
\end{array}
\right]
\end{equation}
we numerate the rows of $X$ starting at $0$, the same we do for the columns of $X$.

2. We compute reduced costs given by $\bar{c}_j = c_j - p^T A_j$, where $p = c_\beta^T B^{-1}$ and we obtain the index of the smallest reduced cost.
\begin{equation}
j = \underset{l \in\{1,\ldots,n+m\}}{\argmin}{\bar{c}_l}
= \underset{l \in\{1,\ldots,n+m\}}{\argmin}{X_{0l}}
\end{equation}

3. If $\bar{c}_j < 0$ we can improve the value of the objective function we proceed with the step 4, otherwise we stop the algorithm and return the 
optimal point.
\begin{equation}
x^* = x_\beta 
= \left[ x_{\beta(1)}, \ldots, x_{\beta(m)} \right]^T
= \left[ X_{10}, \ldots, X_{m0} \right]^T
\end{equation}
and the value of objective function at $x^*$
\begin{equation}
z^* = c_{\beta}^T x_{\beta} = X_{00}
\end{equation}

4. We find the index of the pivot row which is the optimal value for
\begin{equation}
i = \underset{k \in\{1,\ldots,m\}, u_{k} > 0}{\argmin}{\frac{x_{\beta(k)}}{u_{k}}}
= \underset{k \in\{1,\ldots,m\}, X_{kj} > 0}{\argmin}{\frac{X_{0j}}{X_{kj}}}
\end{equation}
in the fir equality $u = B^{-1} A_j$

5. Update the basic bases $\beta = \beta \setminus\{i\} \cup \{j\}$.

6. We update the simplex table by employing the following pivot rules
\begin{equation}
\begin{array}{ll}
X_{il} = \frac{1}{X_{ij}} X_{il} & \text{if}\ k = i \\
X_{kl} = X_{kl} - X_{kj} X_{il} & \text{if}\ k \neq i, k \in \{0,\ldots,m\}
\end{array}
\end{equation}

7. We return to the step 2

In [2]:
# Implementation of simplex algorithm
# Problem considered in the standard form
# minimize c'x subject to Ax <= b
def simplex_algorithm( top, c, A, b ) :
  [ m, n ] = A.shape

  # Reduced costs
  if top == 'max' :
    cr = -c
  elif top == 'min' :
    cr = c
  
  cr = np.insert( cr, 0, 0, axis = 0 )

  # Preparing the simplex table
  X = np.insert( A, 0, b, axis = 1 )
  X = np.insert( X, 0, cr, axis = 0 )

  # Slack variables
  S = np.eye( m )    
  S = np.insert( S, 0, np.zeros( m ), axis = 0 )
  X = np.insert( X, np.repeat( n + 1, m ), S, axis = 1 )
  [ p, q ] = X.shape
  
  # List to save the simplex tables
  ST = []
  ST += [ np.copy( X ) ]
  
  # Feasible base
  Bch = np.arange( q - m, q, 1 )
  B = np.arange( q - m, q, 1 )
  B.shape = [1,m]
  
  cj = -np.inf
  while cj < 0 :

    # Selection pivot column with the smaller reduced cost, following Bland's rule
    cj = X[0,1]
    j = 1
    for l in range( 2, q ) :
      if X[0,l] < cj :
        j = l
        cj = X[0,l]

    # Proceed if the smaller reduced cost is negative
    if cj < 0 :
      theta = np.inf
      i = -1

      # Selecting the pivot row with the smallest ratio x_{B(i)} / u_i
      # If exists more than one we select the smallest index
      for k in range( 1, p ) :
        if X[k,j] > 0 :
          a = X[k,0] / X[k,j] 
          if a >= 0 and a < theta:
            theta = a
            i = k

      # Updating evolution of feasible base
      Bch[ i - 1 ] = j - 1
      B = np.insert( B, B.shape[0], Bch, axis = 0 )
      
      r = 1.0 / X[i,j]
      X[i,:] = r * X[i,:]

      # Pivoting
      for k in range( 0, p ) :
        if k != i :
          r = X[k,j]
          X[k,:] = X[k,:] - r * X[i,:]
      
      ST += [ np.copy( X ) ]

  # Collecting optimal solution in a dictionary for each coordinate
  x = { i : 0 for i in range( 0, q - 1 ) }
  for i in range( 0, m ) :
    x[ Bch[i] ] = X[ i + 1, 0 ]
    
  # Collecting optimal dual solution in a dictionary for each coordinate
  p = { i : 0 for i in range( 0, q - 1 ) }
  for i in range( 1, q ) :
    p[ i - 1 ] = X[ 0, i ]
  
  # Left upper corner of simplex table has the value of the objective function
  if top == 'min' :
    z = -X[0,0]
  elif top == 'max' :
    z = X[0,0]
  
  return [ x, p, z, ST, B ]

## **Ejemplo 1**

In [3]:
n = 4
A = np.eye( n, dtype = np.double )
b = np.ones( n, dtype = np.double )
c = np.ones( n, dtype = np.double )

ST = simplex_algorithm( 'max', c, A, b )

In [4]:
print( 'Optimal point:\n', ST[0] )

Optimal point:
 {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 0, 5: 0, 6: 0, 7: 0}


In [5]:
print( 'Optimal dual:\n', ST[1] )

Optimal dual:
 {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0, 4: 1.0, 5: 1.0, 6: 1.0, 7: 1.0}


In [6]:
print( 'Optimal value:\n', ST[2] ) 

Optimal value:
 4.0


In [7]:
print( 'Evolution of simplex tables:\n' )
for i in range( 0, len( ST[3] ) ) :
  print( ST[3][i], '\n' )

Evolution of simplex tables:

[[ 0. -1. -1. -1. -1.  0.  0.  0.  0.]
 [ 1.  1.  0.  0.  0.  1.  0.  0.  0.]
 [ 1.  0.  1.  0.  0.  0.  1.  0.  0.]
 [ 1.  0.  0.  1.  0.  0.  0.  1.  0.]
 [ 1.  0.  0.  0.  1.  0.  0.  0.  1.]] 

[[ 1.  0. -1. -1. -1.  1.  0.  0.  0.]
 [ 1.  1.  0.  0.  0.  1.  0.  0.  0.]
 [ 1.  0.  1.  0.  0.  0.  1.  0.  0.]
 [ 1.  0.  0.  1.  0.  0.  0.  1.  0.]
 [ 1.  0.  0.  0.  1.  0.  0.  0.  1.]] 

[[ 2.  0.  0. -1. -1.  1.  1.  0.  0.]
 [ 1.  1.  0.  0.  0.  1.  0.  0.  0.]
 [ 1.  0.  1.  0.  0.  0.  1.  0.  0.]
 [ 1.  0.  0.  1.  0.  0.  0.  1.  0.]
 [ 1.  0.  0.  0.  1.  0.  0.  0.  1.]] 

[[ 3.  0.  0.  0. -1.  1.  1.  1.  0.]
 [ 1.  1.  0.  0.  0.  1.  0.  0.  0.]
 [ 1.  0.  1.  0.  0.  0.  1.  0.  0.]
 [ 1.  0.  0.  1.  0.  0.  0.  1.  0.]
 [ 1.  0.  0.  0.  1.  0.  0.  0.  1.]] 

[[4. 0. 0. 0. 0. 1. 1. 1. 1.]
 [1. 1. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 1. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 1. 0. 0. 0. 1.]] 



In [8]:
print( 'Evolution of the feasible base:\n', ST[4] )

Evolution of the feasible base:
 [[5 6 7 8]
 [0 6 7 8]
 [0 1 7 8]
 [0 1 2 8]
 [0 1 2 3]]


## **Ejemplo 2**

In [9]:
A = np.array( [ [ 1, 2, 2 ], [ 2, 1, 2 ], [ 2, 2, 1 ] ], dtype = np.double )
b = np.array( [ 20, 20, 20 ], dtype = np.double )
c = np.array( [ -10, -12, -12 ], dtype = np.double )

ST = simplex_algorithm( 'min', c, A, b )

In [10]:
print( 'Optimal point:\n', ST[0] )

Optimal point:
 {0: 4.0, 1: 4.0, 2: 4.0, 3: 0, 4: 0, 5: 0}


In [11]:
print( 'Optimal dual:\n', ST[1] )

Optimal dual:
 {0: 0.0, 1: 0.0, 2: 0.0, 3: 3.6, 4: 1.6, 5: 1.5999999999999996}


In [12]:
print( 'Optimal value:\n', ST[2] )

Optimal value:
 -136.0


In [13]:
print( 'Evolution of simplex tables:\n' )
for i in range( 0, len( ST[3] ) ) :
  print( ST[3][i], '\n' )

Evolution of simplex tables:

[[  0. -10. -12. -12.   0.   0.   0.]
 [ 20.   1.   2.   2.   1.   0.   0.]
 [ 20.   2.   1.   2.   0.   1.   0.]
 [ 20.   2.   2.   1.   0.   0.   1.]] 

[[120.   -4.    0.    0.    6.    0.    0. ]
 [ 10.    0.5   1.    1.    0.5   0.    0. ]
 [ 10.    1.5   0.    1.   -0.5   1.    0. ]
 [  0.    1.    0.   -1.   -1.    0.    1. ]] 

[[120.    0.    0.   -4.    2.    0.    4. ]
 [ 10.    0.    1.    1.5   1.    0.   -0.5]
 [ 10.    0.    0.    2.5   1.    1.   -1.5]
 [  0.    1.    0.   -1.   -1.    0.    1. ]] 

[[136.    0.    0.    0.    3.6   1.6   1.6]
 [  4.    0.    1.    0.    0.4  -0.6   0.4]
 [  4.    0.    0.    1.    0.4   0.4  -0.6]
 [  4.    1.    0.    0.   -0.6   0.4   0.4]] 



In [14]:
print( 'Evolution of the feasible base:\n', ST[4] )

Evolution of the feasible base:
 [[4 5 6]
 [1 5 6]
 [1 5 0]
 [1 2 0]]
